In [ ]:
<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/04-RAG_with_VectorStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables

In [23]:
!pip install -q llama-index==0.10.5 openai==1.12.0 cohere==4.47 tiktoken==0.6.0 chromadb==0.4.22

In [24]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"

# Load the Dataset (CSV)

## Download

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model. Read the dataset as a long string.

In [25]:
!curl -o ./mini-dataset.csv https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0   743k      0 --:--:-- --:--:-- --:--:--  747k


## Read File

In [26]:
import csv

text = ""

# Load the file as a JSON
with open("./mini-dataset.csv", mode="r", encoding="ISO-8859-1") as file:
  csv_reader = csv.reader(file)

  for row in csv_reader:
    text += row[0]

# The number of characters in the dataset.
len( text )

841

# Chunking

In [27]:
chunk_size = 512
chunks = []

# Split the long text into smaller manageable chunks of 512 characters.
for i in range(0, len(text), chunk_size):
    chunks.append(text[i:i + chunk_size])

len( chunks )

2

In [28]:
from llama_index.core import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=t) for t in chunks]

# Save on Chroma

In [29]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./mini-chunked-dataset")
chroma_collection = chroma_client.create_collection("mini-chunked-dataset")

In [30]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [31]:
from llama_index.core import VectorStoreIndex

# Add the documents to the database and create Index / embeddings
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

# Query Dataset

In [32]:
from llama_index.llms.openai import OpenAI
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.

llm = OpenAI(temperature=0, model="gpt-3.5-turbo-0125", max_tokens=512)
query_engine = index.as_query_engine(llm=llm)

In [33]:
response = query_engine.query(
    "How many parameters LLaMA2 model has?"
)
print(response)

The LLaMA2 model has 7 billion parameters.
